In [1]:
cd ../..

/Users/dorislee/Desktop/Research/vizdatasci/datasci-api


In [2]:
%load_ext autoreload
%autoreload 2
from dataset.Dataset import Dataset
from dataObj.dataObj import DataObj
from dataObj.Row import Row
from dataObj.Column import Column

### Load College dataset

In [3]:
dataset = Dataset("data/college.csv",schema=[{"Year":{"dataType":"date"}}])

In [4]:
dataset.df.head(3)

,Name,PredominantDegree,HighestDegree,FundingModel,Region,Geography,AdmissionRate,ACTMedian,SATAverage,AverageCost,Expenditure,AverageFacultySalary,MedianDebt,AverageAgeofEntry,MedianFamilyIncome,MedianEarnings
0,Alabama A & M University,Bachelor's,Graduate,Public,Southeast,Mid-size City,0.8989,17,823,18888,7459,7079,19500.0,20.629999,29039.0,27000
1,University of Alabama at Birmingham,Bachelor's,Graduate,Public,Southeast,Mid-size City,0.8673,25,1146,19990,17208,10170,16250.0,22.670000,34909.0,37200
2,University of Alabama in Huntsville,Bachelor's,Graduate,Public,Southeast,Mid-size City,0.8062,26,1180,20306,9352,9341,16500.0,23.190001,39766.0,41500


### Look at the relationship between AdmissionRate and other measure values, ranked by correlation

In [5]:
dobj = DataObj(dataset,[Column("?",dataModel="measure"),Column("AdmissionRate")])

In [6]:
# dobj.compiled.collection

In [7]:
dobj.correlation()

In [8]:
widget = dobj.compiled.topK(10).display()

In [9]:
widget

Mockup(numGraphs=9)

#### The relationship between AdmissionRate and Expenditure, SATAverage and AverageFacultySalary is interesting

In [10]:
from utils.postprocessing import *
bookmarkedObj = getSelectedDobjs(dobj,widget)
bookmarkedObj

<Data Obj Collection: [<Data Obj: [Column <Expenditure,channel:x,dataType:quantitative,dataModel:measure>, Column <AdmissionRate,channel:y,dataType:quantitative,dataModel:measure>] -- 0.46>, <Data Obj: [Column <SATAverage,channel:x,dataType:quantitative,dataModel:measure>, Column <AdmissionRate,channel:y,dataType:quantitative,dataModel:measure>] -- 0.37>, <Data Obj: [Column <AverageFacultySalary,channel:x,dataType:quantitative,dataModel:measure>, Column <AdmissionRate,channel:y,dataType:quantitative,dataModel:measure>] -- 0.33>]>

In [11]:
widget = bookmarkedObj.display()
widget

Mockup(numGraphs=3)

#### We are interested in the SATAverage and AdmissionRate, but points are plotted to densely

In [13]:
showJSONspec(widget._graph_specs[widget.selected_graphID[0]])

Add `"mark": {"tooltip": {"content": "encoding"},"opacity": 0.2}` to change the opacity of the points.

In [14]:
admission_vs_salary = getSelectedDobjs(dobj,widget).collection[0]

In [15]:
admission_vs_salary.spec

[Column <SATAverage,channel:x,dataType:quantitative,dataModel:measure>,
 Column <AdmissionRate,channel:y,dataType:quantitative,dataModel:measure>]

### What is relationship between SATAverage and AdmissionRate when broken down by different regions?

In [16]:
admission_vs_salary.spec.append(Row("Region","?"))
admission_vs_salary.compile()

In [17]:
admission_vs_salary.spec

[Column <SATAverage,channel:x,dataType:quantitative,dataModel:measure>,
 Column <AdmissionRate,channel:y,dataType:quantitative,dataModel:measure>,
 Row <Region,?>]

In [18]:
admission_vs_salary.display()

Mockup(numGraphs=8)

### Which regions exhibit the strongest pattern v.s. scattered trend?

In [19]:
admission_vs_salary.correlation()

In [20]:
topK_admission_vs_salary_by_region_widget = admission_vs_salary.compiled.topK(3).display()
topK_admission_vs_salary_by_region_widget

Mockup(numGraphs=3)

In [21]:
bottomK_admission_vs_salary_by_region_widget = admission_vs_salary.compiled.bottomK(3).display()
bottomK_admission_vs_salary_by_region_widget

Mockup(numGraphs=3)

In [22]:
def prPurple(skk): return "\033[95m {}\033[00m".format(skk)
def prGreen(skk): return "\033[92m {}\033[00m".format(skk)

In [23]:
topKschools = prPurple(", ".join(admission_vs_salary.compiled.topK(3).map(lambda x: x.getObjByRowColType("Row")[0].fVal)))
bottomKschools = prPurple(", ".join(admission_vs_salary.compiled.bottomK(3).map(lambda x: x.getObjByRowColType("Row")[0].fVal)))
attributes = prGreen("AdmissionRate and SATAverage")
print(f"There is a strong correlation between {attributes} for colleges in {topKschools} and less so for colleges in {bottomKschools}.")

There is a strong correlation between  AdmissionRate and SATAverage for colleges in  New England, Mid-Atlantic, Rocky Mountains and less so for colleges in  Southeast, Great Plains, Southwest.
